In [1]:
%load_ext autoreload
%autoreload 2

# Librerías

In [2]:
import numpy as np
import timber_nds.essentials as essentials
import timber_nds.design as design
from timber_nds.calculation import WeightCalculator, effective_length, radius_of_gyration, polar_moment_of_inertia, RectangularSectionProperties

# Parámetros


A continuación, se definen todos los parámetros necesarios para el cuaderno, incluyendo factores de reducción de resistencia, propiedades de materiales y dimensiones geométricas.

**Factores de Reducción de Resistencia:**

*   `phi_tension`: Factor de reducción de resistencia para tensión.
*   `phi_bending`: Factor de reducción de resistencia para flexión.
*   `phi_shear`: Factor de reducción de resistencia para corte.
*   `phi_compression`: Factor de reducción de resistencia para compresión.
*    `phi_perp_compression`: Factor de reducción de resistencia para compresión perpendicular a la fibra.

**Propiedades del Material:**

*   `species`: Especie de madera.
*   `specific_gravity`: Gravedad específica de la madera.
*   `fibre_saturation_point`: Punto de saturación de la fibra de la madera (%).
*   `moisture_content`: Contenido de humedad de la madera (%).
*   `e_modulus`: Módulo de elasticidad de la madera (Pa).
*  `fb`: Resistencia a la flexión (Pa).
* `ft`: Resistencia a la tracción (Pa).
* `fv`: Resistencia al corte (Pa).
* `fc`: Resistencia a la compresión paralela a la fibra (Pa).
* `fc_perp`: Resistencia a la compresión perpendicular a la fibra (Pa).

**Dimensiones Geométricas:**

*   `width`: Ancho de la sección transversal (m).
*   `depth`: Profundidad de la sección transversal (m).
*   `length`: Longitud del elemento (m).
*   `k_factor`: Factor de longitud efectiva.

**Factores de Ajuste:**

*   Se definen varios factores de ajuste para tensión, flexión, corte, compresión, compresión perpendicular a la fibra y módulo de elasticidad.


In [3]:

# Factors
phi_tension = 0.90
phi_bending = 0.85
phi_shear = 0.75
phi_compression = 0.90
phi_perp_compression = 0.65

# Material Properties
species = "Douglas Fir-Larch"
specific_gravity = 0.55
fibre_saturation_point = 28
moisture_content = 19
e_modulus = 115000 # kgf/cm2
fb = 280  # kgf/cm2
ft = 280  # kgf/cm2a
fv = 45  # kgf/cm2
fc = 280  # kgf/cm2
fc_perp = 28  # kgf/cm2


# Geometry
width = 15.0 # cm
depth = 30. # cm
length = 4.0 # m
k_factor = 1.0

# Adjustment factors initialization
ft_factors = essentials.TensionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_yy = essentials.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fb_factors_zz = essentials.BendingAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.85,
    due_temperature=1.0,
    due_size=1.0,
    due_beam_stability=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fv_factors = essentials.ShearAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.97,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_yy = essentials.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_factors_zz = essentials.CompressionAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.80,
    due_temperature=1.0,
    due_size=1.0,
    due_incising=1.0,
    due_column_stability=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

fc_perp_factors = essentials.PerpendicularAdjustmentFactors(
    due_time_effect=1.0,
    due_moisture=0.67,
    due_temperature=1.0,
    due_beam_stability=1.0,
    due_size=1.0,
    due_flat_use=1.0,
    due_incising=1.0,
    due_repetitive_member=1.0,
    due_column_stability=1.0,
    due_buckling_stiffness=1.0,
    due_bearing_area=1.0,
    due_format_conversion=1.0,
    due_resistance_reduction=1.0
)

e_factors = essentials.ElasticModulusAdjustmentFactors(
    due_moisture=0.90,
    due_temperature=1.0,
    due_incising=1.0,
    due_format_conversion=1.0
)

# Ejemplo


Este cuaderno demostrará el cálculo del peso, las propiedades geométricas y la capacidad de una viga de madera de sección rectangular. 
Comenzaremos calculando el peso de la viga, luego sus propiedades geométricas, y finalmente, integraremos todos los resultados para calcular la capacidad de la viga.
El ejemplo se desarrollará de manera continua a través de todas las secciones, ilustrando el flujo de trabajo típico en el diseño de estructuras de madera.


## Ejemplo de Función 1: Cálculo del Peso


Esta sección demuestra cómo calcular el peso de un elemento de madera utilizando la clase `WeightCalculator`.
Primero, se crea una instancia de `WoodMaterial`, `RectangularSection` y `MemberDefinition` con los parámetros definidos anteriormente.
Luego, se crea una instancia de `WeightCalculator` y se calcula el peso del elemento para el contenido de humedad especificado.


In [4]:

# Creating instances of classes
wood_material = essentials.WoodMaterial(
    name=species,
    specific_gravity=specific_gravity,
    fibre_saturation_point=fibre_saturation_point,
    tension_strength=ft,
    bending_strength=fb,
    shear_strength=fv,
    compression_perpendicular_strength=fc_perp,
    compression_parallel_strength=fc,
    elastic_modulus=e_modulus,
    color="Brown"
)

section = essentials.RectangularSection(depth=200)
element = essentials.MemberDefinition(length, )
weight_calculator = WeightCalculator(wood_material, section, element)
element_weight = weight_calculator.calculate_weight_at_moisture_content(moisture_content)

print(f"El peso del elemento de madera es: {element_weight:.2f} kg")
    

El peso del elemento de madera es: 34029462738.30 kg


## Ejemplo de Función 2: Propiedades Geométricas


En esta sección, calculamos las propiedades geométricas de la sección rectangular utilizando la clase `RectangularSectionProperties`.
Se crea una instancia de `RectangularSectionProperties` con el ancho y la profundidad de la sección.
Luego, calculamos el área, los momentos de inercia, los módulos de sección y los radios de giro.


In [5]:

# Calculate Section Properties
section_properties = RectangularSectionProperties(width, depth)
area = section_properties.area()
i_yy = section_properties.moment_of_inertia("yy")
i_zz= section_properties.moment_of_inertia("zz")
elastic_section_modulus_yy = section_properties.elastic_section_modulus("yy")
elastic_section_modulus_zz = section_properties.elastic_section_modulus("zz")
radius_gyration_yy = section_properties.radius_of_gyration("yy")
radius_gyration_zz = section_properties.radius_of_gyration("zz")

print(f"Área de la sección: {area:.1f} cm^2")
print(f"Momento de inercia Iyy: {i_yy:.1f} cm^4")
print(f"Momento de inercia Izz: {i_zz:.1f} cm^4")
print(f"Módulo de sección elástico Sxx: {elastic_section_modulus_yy:.1f} cm^3")
print(f"Módulo de sección elástico Syy: {elastic_section_modulus_zz:.1f} cm^3")
print(f"Radio de giro r_yy: {radius_gyration_yy:.1f} cm")
print(f"Radio de giro r_zz: {radius_gyration_zz:.1f} cm")
    

Área de la sección: 450.0 cm^2
Momento de inercia Iyy: 33750.0 cm^4
Momento de inercia Izz: 8437.5 cm^4
Módulo de sección elástico Sxx: 2250.0 cm^3
Módulo de sección elástico Syy: 1125.0 cm^3
Radio de giro r_yy: 8.7 cm
Radio de giro r_zz: 4.3 cm


## Ejemplo de Función 3: Longitud Efectiva


Aquí, calculamos la longitud efectiva de la viga usando la función `effective_length`.
La longitud efectiva se calcula multiplicando la longitud real de la viga por el factor de longitud efectiva (K).


In [6]:

# Calculate Effective Length
effective_length_value = effective_length(k_factor, length)
print(f"Longitud efectiva de la viga: {effective_length_value:.2f} m")
    

Longitud efectiva de la viga: 4.00 m


## Ejemplo de Función 4: Factores Combinados


En esta sección, demostramos cómo utilizar la clase `WoodElementCalculator` para calcular los factores combinados.
Se crea una instancia de `WoodElementCalculator` con todos los factores de ajuste definidos anteriormente.
Luego, se calcula el factor combinado para cada propiedad mecánica (tensión, flexión, corte, compresión, compresión perpendicular a la fibra y módulo de elasticidad).


In [7]:
# Crear una instancia de WoodElementCalculator
wood_element_calculator = design.WoodElementCalculator(
    tension_factors = ft_factors,
    bending_factors_yy = fb_factors_yy,
    bending_factors_zz = fb_factors_zz,
    shear_factors = fv_factors,
    compression_factors_yy = fc_factors_yy,
    compression_factors_zz = fc_factors_zz,
    compression_perp_factors = fc_perp_factors,
    elastic_modulus_factors = e_factors,
    material_properties = wood_material,
    section_properties = section_properties
)

# Calcular los factores combinados
combined_factors = wood_element_calculator.calculate_combined_factors()
print(f"Factores combinados: {combined_factors}")
    

Factores combinados: {'tension': np.float64(0.85), 'bending_yy': np.float64(0.85), 'bending_zz': np.float64(0.85), 'shear': np.float64(0.97), 'compression_yy': np.float64(0.8), 'compression_zz': np.float64(0.8), 'compression_perp': np.float64(0.67), 'elastic_modulus': np.float64(0.9)}


## Ejemplo de Función 5: Resistencia del elemento

In [8]:
tension_strength = wood_element_calculator.section_tension_strength()
bending_strength_yy = wood_element_calculator.section_bending_strength("yy")
bending_strength_zz = wood_element_calculator.section_bending_strength("zz")
shear_strength = wood_element_calculator.section_shear_strength()
compression_strength_yy = wood_element_calculator.section_compression_strength("yy")
compression_strength_zz = wood_element_calculator.section_compression_strength("zz")
compression_perp_strength = wood_element_calculator.section_compression_perp_strength()


In [10]:
print(f'La resistencia a la tensión es {float(tension_strength)} kgf')
print(f'La resistencia a la flexión es {float(bending_strength_yy)} kgf cm')
print(f'La resistencia a la flexión es {float(bending_strength_zz)} kgf cm')
print(f'La resistencia al cortante es {float(shear_strength)} kgf')
print(f'La resistencia a la compresión yy es {float(compression_strength_yy)} kgf')
print(f'La resistencia a la compresión zz es {float(compression_strength_zz)} kgf')
print(f'La resistencia a la compresión perpendicular es {float(compression_perp_strength)} kgf')

La resistencia a la tensión es 107100.0 kgf
La resistencia a la flexión es 535500.0 kgf cm
La resistencia a la flexión es 267750.0 kgf cm
La resistencia al cortante es 13095.0 kgf
La resistencia a la compresión yy es 504000.0 kgf
La resistencia a la compresión zz es 252000.0 kgf
La resistencia a la compresión perpendicular es 84420.0 kgf


# Pruebas


### Prueba para `WeightCalculator.calculate_density_at_moisture_content`

```python
import pytest
from timber_nds.calculation import WeightCalculator
import timber_nds.essentials as essentials

def test_calculate_density_at_moisture_content():
    # Arrange
    wood_material = essentials.WoodMaterial(
        species = "Douglas Fir-Larch",
        specific_gravity = 0.50,
        fibre_saturation_point = 28,
        e_modulus = 12400000000,
        fb = 25000000,
        ft = 15000000,
        fv = 3000000,
        fc = 17000000,
        fc_perp = 4000000
    )
    section = essentials.RectangularSection(0.150, 0.300)
    element = essentials.MemberDefinition(4)
    weight_calculator = WeightCalculator(wood_material, section, element)
    moisture_content = 19

    # Act
    density = weight_calculator.calculate_density_at_moisture_content(moisture_content)

    # Assert
    assert density == pytest.approx(576.13, 0.01)

def test_calculate_density_at_moisture_content_invalid_input():
        # Arrange
    wood_material = essentials.WoodMaterial(
        species = "Douglas Fir-Larch",
        specific_gravity = 0.50,
        fibre_saturation_point = 28,
        e_modulus = 12400000000,
        fb = 25000000,
        ft = 15000000,
        fv = 3000000,
        fc = 17000000,
        fc_perp = 4000000
    )
    section = essentials.RectangularSection(0.150, 0.300)
    element = essentials.MemberDefinition(4)
    weight_calculator = WeightCalculator(wood_material, section, element)


    with pytest.raises(TypeError):
        weight_calculator.calculate_density_at_moisture_content("invalid")
    with pytest.raises(ValueError):
        weight_calculator.calculate_density_at_moisture_content(-5)
    with pytest.raises(ValueError):
       wood_material_negative_fsp = essentials.WoodMaterial(
            species = "Douglas Fir-Larch",
            specific_gravity = 0.50,
            fibre_saturation_point = -28,
             e_modulus = 12400000000,
            fb = 25000000,
            ft = 15000000,
            fv = 3000000,
            fc = 17000000,
            fc_perp = 4000000
        )
       weight_calculator_negative_fsp = WeightCalculator(wood_material_negative_fsp, section, element)
       weight_calculator_negative_fsp.calculate_density_at_moisture_content(10)
